In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb

from sklearn.model_selection import train_test_split, StratifiedKFold
#from sklearn.metrics import cohen_kappa_score, accuracy_score,balanced_accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import KFold # Use KFold for regression
from sklearn.metrics import mean_squared_error


#from plotly import express as px

#from utils import plot_confusion_matrix

import os

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

from joblib import load, dump
import pyarrow.parquet as pq


import gc
import shutil

## Armado Dataset

In [2]:
# Paths
BASE_DIR = '../'

#PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train.parquet")
#PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train_producto.parquet")
PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/1_c_producto_train.parquet")
PATH_TO_MODELS = os.path.join(BASE_DIR, "LGBM/models")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "LGBM/optuna_/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "LGBM/optuna_/optuna_artifacts")
PATH_TO_MODELS_OLD=PATH_TO_MODELS_OLD = os.path.join(BASE_DIR, "LGBM/models/old")


nombrebase="sqlite:///db.sqlite10115"
nombreestudio="producto_sindtw_sin_prophet_estrategia_test_adecuada"

SEED = 42
BATCH_SIZE = 50
#TEST_SIZE = 0.2
PATH_TO_TRAIN

'../Dataset/1_c_producto_train.parquet'

In [3]:
dataset = pd.read_parquet(PATH_TO_TRAIN)

In [4]:
del dataset['ultima_tn']
del dataset['cluster_500']
del dataset['cluster_50']
del dataset['cluster_100']
del dataset['cluster_10']
del dataset['cluster_3']
del dataset['cluster_2']
del dataset['prod_season_yearly']
del dataset['cat_season_yearly']
del dataset['total_season_yearly']
del dataset['prod_trend']
del dataset['cat_trend']
del dataset['total_trend']

In [5]:
dataset.shape

(29652, 299)

In [6]:
char_feats = dataset.select_dtypes(include=['object', 'category']).columns.tolist()
print(char_feats)
#numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

['cat1', 'cat2', 'cat3', 'brand', 'categoria', 'estado_producto']


In [7]:
numeric_feats = dataset.select_dtypes(include=['number']).columns.tolist()
print(numeric_feats)

['periodo', 'product_id', 'producto_total_tn', 'avg_tn', 'std_tn', 'clientes_distintos', 'cust_request_qty', 'cust_request_tn', 'inicio_vida_p', 'fin_vida_p', 'sku_size', 'stock_final', 'dias_mes_4', 'dias_mes_3', 'anio', 'mes', 'trimestre', 'outlier', 'outlier-2', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6', 'tn_lag_7', 'tn_lag_8', 'tn_lag_9', 'tn_lag_10', 'tn_lag_11', 'tn_lag_12', 'tn_lag_13', 'tn_lag_14', 'tn_lag_15', 'tn_lag_16', 'tn_lag_17', 'tn_lag_18', 'tn_lag_19', 'tn_lag_20', 'tn_lag_21', 'tn_lag_22', 'tn_lag_23', 'tn_lag_24', 'tn_lag_25', 'tn_lag_26', 'tn_lag_27', 'tn_lag_28', 'tn_lag_29', 'tn_lag_30', 'tn_lag_31', 'tn_lag_32', 'tn_lag_33', 'tn_lag_34', 'tn_lag_35', 'tn_lag_36', 'tn_media_movil_3', 'tn_media_movil_6', 'tn_media_movil_9', 'tn_media_movil_12', 'tn_std_movil_3', 'tn_std_movil_6', 'tn_std_movil_9', 'tn_std_movil_12', 'tn_min_movil_3', 'tn_min_movil_6', 'tn_min_movil_9', 'tn_min_movil_12', 'delta_media_movil_12', 'delta_media_movil_3', '

In [8]:
dataset.shape

(29652, 299)

In [9]:
gc.collect()

39

In [10]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        #   - Datetime
        # 1) Datetime → int64 (ns) → float32
        if pd.api.types.is_datetime64_any_dtype(col_type):
            # view() extrae los nanosegundos desde epoch
            df[col] = df[col].view('int64').astype('float32')
            continue


        # Solo nos ocupamos de numéricos
        if not pd.api.types.is_numeric_dtype(col_type):
            if not pd.api.types.is_categorical_dtype(col_type):
                df[col] = df[col].astype('category')
            continue

        c_min, c_max = df[col].min(), df[col].max()
        has_na = df[col].isnull().any()

        # --- ENTEROS ---
        if pd.api.types.is_integer_dtype(col_type):
            # 1) Sin nulos → numpy ints
            if not has_na:
                if c_min >= np.iinfo(np.int8).min  and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)


        # --- FLOTANTES ---
        else:
            df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')
        print(f'Uso de memoria final del DataFrame:   {end_mem:.2f} MB')
        print(f'Memoria reducida en un {(100*(start_mem-end_mem)/start_mem):.2f}%')
    return df

In [11]:
dataset = reduce_mem_usage(dataset)

Uso de memoria inicial del DataFrame: 17.98 MB
Uso de memoria final del DataFrame:   32.35 MB
Memoria reducida en un -79.89%


C:\Users\diana\AppData\Local\Temp\ipykernel_8684\4128757994.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):
C:\Users\diana\AppData\Local\Temp\ipykernel_8684\4128757994.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):
C:\Users\diana\AppData\Local\Temp\ipykernel_8684\4128757994.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):
C:\Users\diana\AppData\Local\Temp\ipykernel_8684\4128757994.py:11: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the 

In [12]:
gc.collect()

0

In [13]:
dataset.shape

(29652, 299)

In [14]:
dataset = dataset.sort_values('periodo')

periodos = dataset['periodo'].unique()

periodos_train = periodos[:-2]
periodos_test  = periodos[-2:]

train = dataset[dataset['periodo'].isin(periodos_train)]
test  = dataset[dataset['periodo'].isin(periodos_test)]


In [15]:
print(dataset['periodo'].unique())

[201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201908 201909 201910]


In [16]:
print(train['periodo'].unique())

[201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201908]


In [17]:
print(test ['periodo'].unique())

[201909 201910]


In [18]:
train.shape

(27764, 299)

In [19]:
# Eliminar el DataFrame
del dataset

gc.collect()

0

In [20]:
label = 'clase_producto'
features = [col for col in train.columns if col != label]

In [21]:
train.to_parquet("train_tmp.parquet", index=False)
del train
gc.collect()

train = pd.read_parquet("train_tmp.parquet")


In [22]:
test.to_parquet("test_tmp.parquet", index=False)
del test
gc.collect()

test = pd.read_parquet("test_tmp.parquet")

In [23]:
train.shape

(27764, 299)

In [24]:
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [25]:
X_train.shape

(27764, 298)

In [26]:
del train
#del test

gc.collect()

0

In [27]:
print("X_train shape:", X_train.shape)

X_train shape: (27764, 298)


## Entrenamiento

In [28]:
def lgb_objective(trial):

    # Parámetros para LightGBM
    lgb_params = {
        'objective': 'regression',
        'metric': 'mse', # Cambiado a Mean Squared Error
        'verbosity': -1,
        'seed': SEED,
        # 'num_class': len(y_train.unique()), # Eliminar, esto es para clasificación
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 8, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_bin': trial.suggest_int('max_bin', 64, 1024),
    }

    # Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    # Para regresión, scores_ensemble será un array 1D
    scores_ensemble = np.zeros(len(y_test),dtype=np.float32)

    # Score del 5 fold CV inicializado en 0
    score_folds = 0

    # Numero de splits del CV
    n_splits = 5

    # Objeto para hacer el split de CV (usar KFold para regresión, ya que StratifiedKFold es para clasificación)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED) # Added shuffle and random_state for reproducibility
    best_iterations = []  # ⬅️ para acumular las best_iteration de cada fold

    for i, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)): # Usar kf.split

        # Dataset in fold (donde entreno)
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                     label=y_train.iloc[if_index],
                                     free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                      label=y_train.iloc[oof_index],
                                      free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                              lgb_if_dataset,
                              valid_sets=lgb_oof_dataset,
                              num_boost_round=10000,
                              callbacks=[lgb.early_stopping(200, verbose=False)],
                              # feval = mean_squared_error(y_test, preds, squared=False) # Eliminar o definir correctamente para custom metric
                             )

        # Acumulo las predicciones continuas para el conjunto de test
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)

        # Score del fold (registros de dataset train que en este fold quedan out of fold)
        # Calcular MSE para el fold OOF
        oof_preds = lgb_model.predict(X_train.iloc[oof_index])
        score_folds += mean_squared_error(y_train.iloc[oof_index], oof_preds) / n_splits
        
        # guardo el best_iteration de este fold
        best_iterations.append(lgb_model.best_iteration)
        
        # ⬇️ Liberar memoria de objetos pesados del fold
        del lgb_model, lgb_if_dataset, lgb_oof_dataset, oof_preds
        gc.collect()
       #print(f"fin split {i}")

    # Promedio las predicciones del ensemble para el conjunto de test
    scores_ensemble = scores_ensemble / n_splits

    # Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    #predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    #predicted_df = test.copy()
    #predicted_df= pd.DataFrame({
    #'periodo': test['periodo'],
    #'customer_id': test['customer_id'],  # o el ID que necesites
    #'product_id' : test['product_id'],
    #'pred': scores_ensemble
    #})
    
    # Genero columna pred con predicciones promediadas de los 5 folds
    #predicted_df['pred'] = scores_ensemble
    # Grabo dataframe en temp_artifacts
    #dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    #upload_artifact(trial, predicted_filename, artifact_store)
    #del predicted_df
    gc.collect()
    
    # Grabo métricas de regresión en lugar de matriz de confusión
    # Puedes guardar métricas como MSE, RMSE, MAE, R2, etc. en un archivo de texto o log
    # Por ejemplo, calcular el MSE en el conjunto de test
    test_mse = mean_squared_error(y_test, scores_ensemble)
    trial.set_user_attr("test_mse", test_mse) # Almacenar MSE del test como atributo de usuario

    # guardar el promedio de las mejores iteraciones
    avg_best_iteration = int(np.mean(best_iterations))
    trial.set_user_attr("best_iteration", avg_best_iteration)
    
    del scores_ensemble
    gc.collect()

    # Si quieres una visualización, podrías generar un scatter plot de predicciones vs valores reales
    # O un histograma de residuos. Aquí un placeholder para un "regression_metrics.txt"
    #regression_metrics_filename = os.path.join(PATH_TO_TEMP_FILES, f'regression_metrics_{trial.study.study_name}_{trial.number}.txt')
    #with open(regression_metrics_filename, 'w') as f:
    #    f.write(f"Test Mean Squared Error: {test_mse}\n")
    #    # Add other regression metrics if desired
    #upload_artifact(trial, regression_metrics_filename, artifact_store)
    
    gc.collect()

    # Devuelvo el score promedio de MSE del 5-fold CV a Optuna para que optimice en base a eso
    # Optuna minimiza por defecto, por lo que devolver MSE es apropiado.
    return score_folds

In [40]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='minimize',
                            storage=nombrebase,  # Specify the storage URL here.
                            study_name=nombreestudio,
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(lgb_objective, n_trials=4000, gc_after_trial=True)

[I 2025-07-19 21:54:16,660] Using an existing study with name 'producto_sindtw_sin_prophet_estrategia_test_adecuada' instead of creating a new one.
[I 2025-07-19 21:57:43,590] Trial 239 finished with value: 714.3502776949776 and parameters: {'lambda_l1': 0.00070464321892365, 'lambda_l2': 8.210367986809603e-05, 'num_leaves': 18, 'feature_fraction': 0.8620321886672656, 'learning_rate': 0.010149716194385712, 'bagging_fraction': 0.8527325417261208, 'bagging_freq': 1, 'min_child_samples': 5, 'max_bin': 175}. Best is trial 180 with value: 704.7100468330754.
[I 2025-07-19 21:58:15,991] Trial 240 finished with value: 724.4842948127937 and parameters: {'lambda_l1': 0.0014502374803268985, 'lambda_l2': 0.00028495100458157213, 'num_leaves': 8, 'feature_fraction': 0.8731338497084915, 'learning_rate': 0.09178872723723092, 'bagging_fraction': 0.8536360118201987, 'bagging_freq': 1, 'min_child_samples': 12, 'max_bin': 156}. Best is trial 180 with value: 704.7100468330754.
[I 2025-07-19 22:00:05,778] Tr

KeyboardInterrupt: 

## Guardar modelos con mejores parametros

In [41]:
# Buscar el trial con menor mse_test en los atributos de usuario
best_trial = min(
    [t for t in study.trials if t.user_attrs.get("test_mse") is not None],
    key=lambda t: t.user_attrs["test_mse"]
)

In [42]:
print(best_trial)

FrozenTrial(number=498, state=1, values=[740.6580563618578], datetime_start=datetime.datetime(2025, 7, 20, 7, 9, 19, 946002), datetime_complete=datetime.datetime(2025, 7, 20, 7, 13, 45, 22396), params={'lambda_l1': 0.00048780068283068646, 'lambda_l2': 6.841445986287358e-05, 'num_leaves': 16, 'feature_fraction': 0.8279543089006356, 'learning_rate': 0.016214350390850327, 'bagging_fraction': 0.8466171909046838, 'bagging_freq': 1, 'min_child_samples': 5, 'max_bin': 918}, user_attrs={'best_iteration': 4657, 'test_mse': 595.0331742875799}, system_attrs={}, intermediate_values={}, distributions={'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=8, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'bagging_fraction': FloatDist

In [43]:
best_params = best_trial.params.copy()
best_params.update({
    "objective": "regression",#los parametros que no se optimizan (los fijos) hay que escribirlos
    "metric": "mse",
    "verbosity": -1,
    'seed': SEED    
})

In [44]:
print( best_trial.params)

{'lambda_l1': 0.00048780068283068646, 'lambda_l2': 6.841445986287358e-05, 'num_leaves': 16, 'feature_fraction': 0.8279543089006356, 'learning_rate': 0.016214350390850327, 'bagging_fraction': 0.8466171909046838, 'bagging_freq': 1, 'min_child_samples': 5, 'max_bin': 918}


In [45]:
best_iteration = best_trial.user_attrs["best_iteration"]
print( best_trial.params)

{'lambda_l1': 0.00048780068283068646, 'lambda_l2': 6.841445986287358e-05, 'num_leaves': 16, 'feature_fraction': 0.8279543089006356, 'learning_rate': 0.016214350390850327, 'bagging_fraction': 0.8466171909046838, 'bagging_freq': 1, 'min_child_samples': 5, 'max_bin': 918}


In [46]:
print(X_train.shape)
print(X_test.shape)
# Datos completos
X_final = pd.concat([X_train, X_test], axis=0)
print(X_final.shape)

(27764, 298)
(1888, 298)
(29652, 298)


In [47]:
print(y_train.shape)
print(y_test.shape)
y_final = pd.concat([y_train, y_test], axis=0)
print(y_final.shape)

(27764,)
(1888,)
(29652,)


In [48]:
def entrenar_lgbm_final_todo(base_params, seed):
    """
    Entrena un modelo final en TODO el dataset (train+test),
    usando los mejores hiperparámetros y un número fijo de iteraciones.

    Args:
        base_params (dict): hiperparámetros óptimos (Optuna).
        seed (int): semilla para reproducibilidad.

    Returns:
        modelo final entrenado.
    """
    run_params = base_params.copy()
    run_params.update({'seed': seed})

    print("\n🚀 Entrenando modelo final en TODO el dataset...")


    all_dataset = lgb.Dataset(
        data=X_final,
        label=y_final,
        free_raw_data=True,
        categorical_feature=char_feats
    )

    # entrenar con el mejor número de iteraciones conocido
    # si no lo conocés, podés poner 10000, pero sin early stopping (porque no hay validación)
    final_model = lgb.train(
        run_params,
        train_set=all_dataset,
        num_boost_round=best_iteration  # 👈 poné aquí el mejor obtenido antes
    )

    # Guardar modelo
    model_path = os.path.join(PATH_TO_MODELS, f"lgb_final_model_todo_seed_{seed}.joblib")
    dump(final_model, model_path)
    print(f"✅ Modelo final entrenado y guardado en: {model_path}")

    gc.collect()

    return final_model

In [49]:
# Mover modelos previos antes del loop
for fname in os.listdir(PATH_TO_MODELS):
    if fname.endswith('.joblib'):
        src_path = os.path.join(PATH_TO_MODELS, fname)
        dst_path = os.path.join(PATH_TO_MODELS_OLD, fname)
        shutil.move(src_path, dst_path)
        print(f"🔁 Modelo movido a backup: {fname}")

🔁 Modelo movido a backup: lgb_final_model_todo_seed_1.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_10.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1000.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_101.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1024.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1050.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_11.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1100.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_111.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1200.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1234.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_125.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1300.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_135.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed_1357.joblib
🔁 Modelo movido a backup: lgb_final_model_todo_seed

In [50]:
#results = []
#SEEDS = [42,5,915,15,666,9999,37,45,125,90,1000,3,753,159,852,10,7,1,1050,654,11,21,33,69,8008,88,111,222,314,420,512,777,808,999,1024,2048,4096,17,19,23,29,31,25]
SEEDS = [
    42, 5, 915, 15, 666, 9999, 37, 45, 125, 90,
    1000, 3, 753, 159, 852, 10, 7, 1, 1050, 654,
    11, 21, 33, 69, 8008, 88, 111, 222, 314, 420,
    512, 777, 808, 999, 1024, 2048, 4096, 17, 19,
    23, 29, 31, 25,
    1234, 4321, 867, 1357, 2468, 3141, 2718, 7000, 8888, 4444,
    5555, 2021, 1984, 3001, 6006, 9090, 707, 3333, 2222, 1759,
    987, 543, 876, 3456, 7890, 246, 135, 6420, 9990, 888,
    7777, 555, 444, 222, 101, 303, 909, 505, 7070, 8080,
    1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000,
    2121, 2323, 2525, 2727,31337, 4242, 6061
]
for current_seed in SEEDS:
    # <--- CORRECCIÓN 1 y 2: La función ahora se llama con los parámetros base y la semilla actual
    entrenar_lgbm_final_todo(
        base_params=best_params, 
        seed=current_seed
    )


🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_42.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_5.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_915.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_15.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_666.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_9999.joblib

🚀 Entrenando modelo final en TODO el dataset...
✅ Modelo final entrenado y guardado en: ../LGBM/models\lgb_final_model_todo_seed_37.joblib

🚀 Entrenando mod